In [1]:
from scamp import test_run, Session, settings
from config import ARTIFACTS_RELPATH
from scamp_utils import set_default_scamp_settings

set_default_scamp_settings()

ALSA lib conf.c:4005:(snd_config_hooks_call) Cannot open shared library libasound_module_conf_pulse.so (/opt/conda/envs/default/lib/alsa-lib/libasound_module_conf_pulse.so: cannot open shared object file: No such file or directory)
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM default
fluidsynth: error: Failed to open the "default" audio device
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'coreaudio'.
fluidsynth: This build of fluidsynth supports the following audio drivers: alsa, file, jack, oss, pulseaudio, sdl2
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'dsound'.
fluidsynth: This build of fluidsynth supports the following audio drivers: alsa, file, jack, oss, pulseau

Testing for working audio driver...
jackdmp 1.9.22
Copyright 2001-2005 Paul Davis and others.
Copyright 2004-2016 Grame.
Copyright 2016-2023 Filipe Coelho.
jackdmp comes with ABSOLUTELY NO WARRANTY
This is free software, and you are welcome to redistribute it
under certain conditions; see the file COPYING for details


Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
fluidsynth: error: Failed to connect to Jack server.
ALSA lib seq_hw.c:466:(snd_seq_hw_open) open /dev/snd/seq failed: No such file or directory
fluidsynth: error: Error opening ALSA sequencer
fluidsynth: error: Couldn't find the requested audio driver 'portaudio

In [42]:
supercollider_session = Session()
phased_osc = supercollider_session.new_osc_part(
    "phased", port=57120, ip_address="supercollider"
)
mono_osc = supercollider_session.new_osc_part(
    "mono", port=57120, ip_address="supercollider"
)

chord = [60, 64, 67]

for pitch in chord:
    phased_osc.play_note(
        pitch, 0.7, 1, {"param_tremoloFreq": 3}
    )
for pitch in chord:
    mono_osc.play_note(
        pitch, 0.7, 1, {"param_freqVariation": 0.05}
    )